In [68]:
from osgeo import gdal, osr

In [65]:
from PIL import Image

In [10]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [82]:
import matplotlib.image as mpimg

In [3]:
data_date = "2018_06_27_13_18_13"
root_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/"
data_dir = root_dir + data_date

In [5]:
src_filename = root_dir + "n48_w002_1arc_v3.dt2"
dataset = gdal.Open(src_filename, gdal.GA_ReadOnly)

In [6]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))

# Fetch the coefficients for transforming between 
# pixel/line (P,L) raster space => projection coordinates (Xp,Yp) space
# Xp = GT[0] + P*GT[1] + L*GT[2]
# Yp = GT[3] + P*GT[4] + L*GT[5]

GT = dataset.GetGeoTransform()
if GT:
    print("Origin = ({}, {})".format( GT[0], GT[3] ) )
    print("Pixel Size = ({}, {})".format( GT[1], GT[5] ) )

Driver: DTED/DTED Elevation Raster
Size is 3601 x 3601 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
Origin = (-2.000138888888889, 49.000138888888884)
Pixel Size = (0.0002777777777777778, -0.0002777777777777778)


In [7]:
band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

Band Type=Int16


In [13]:
XSize = band.XSize
YSize = band.YSize
DEM = band.ReadAsArray(0, 0, XSize, YSize )

In [31]:
dted_long = GT[0] + np.arange(XSize) * GT[1]
dted_lat = GT[3] + np.arange(YSize) * GT[5]
meshgrid_long, meshgrid_lat = np.meshgrid(dted_long, dted_lat)

In [72]:
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

In [107]:
levels = 10
#plt.contour(DEM, levels,linewidths=0.2,colors='k')
#plt.imshow(DEM,cmap ='RdYlGn_r',origin='lower')
vmin = 0
vmax = 130
plt.imshow(dted_elevations, vmin=vmin, vmax=vmax, cmap='Greys')
plt.savefig(root_dir + 'ContourMapTight.png',dpi=900)

In [29]:
# topLeft => 48.559066 , -1.607419
# bottomRight => 48.460979 , -1.444684
latMax = 48.559066
latMin = 48.460979
longMin = -1.607419
longMax = -1.444684

In [115]:
idxLong = np.where( (dted_long >= longMin) & (dted_long <= longMax) )
idxLat =  np.where( (dted_lat >= latMin) & (dted_lat <= latMax) )
idxLongMin = idxLong[0][0]
idxLongMax = idxLong[0][-1]
idxLatMin = idxLat[0][0]
idxLatMax = idxLat[0][-1]
meshgrid_long_pleineFougeres, meshgrid_lat_pleineFougere = np.meshgrid(dted_long[idxLong], dted_lat[idxLat])

In [161]:
dted_lat[idxLat].shape, dted_long[idxLong].shape, pleineFougeresDEM.shape

((353,), (586,), (353, 586))

In [186]:
levels = 10
vmin = 0
vmax = 130

pleineFougeresDEM = dted_elevations[idxLatMin:idxLatMax+1, idxLongMin:idxLongMax+1]

sizes = np.shape(pleineFougeresDEM)
height = float(sizes[0])
width = float(sizes[1])
     
fig = plt.figure()
mult = 5
fig.set_size_inches(width/height*mult, 1*mult, forward=False)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(pleineFougeresDEM, vmin=vmin, vmax=vmax, cmap='terrain')
plt.contourf(pleineFougeresDEM, 10, vmin=vmin, vmax=vmax, cmap='terrain')
plt.savefig(root_dir + 'pleineFougeres.png',dpi=height)

In [209]:
levels = 10
vmin = 0
vmax = 130
     
fig = plt.figure()
# left right bottom top
extent=( longMin, longMax, latMax, latMin )
#plt.imshow(pleineFougeresDEM, vmin=vmin, vmax=vmax, cmap='terrain', extent=extent)
plt.contourf(pleineFougeresDEM, 10, vmin=vmin, vmax=vmax, cmap='terrain', origin='lower', extent=extent)
plt.title("Pleine Fougères")
plt.grid()
plt.colorbar()

In [210]:
vmin = 0
vmax = 130
plt.figure()
plt.pcolormesh( dted_long, dted_lat, dted_elevations,
               cmap='gray', vmin=vmin, vmax=vmax)
plt.pcolormesh( dted_long[idxLong], dted_lat[idxLat], pleineFougeresDEM,
               cmap='terrain', vmin=vmin, vmax=vmax)

ax = plt.gca()

ax.set_aspect("equal")
plt.colorbar()
plt.grid()

In [129]:
def arrayToRaster(array,fileName,EPSGCode,xMin,xMax,yMin,yMax,numBands):
    xPixels = array.shape[1]  # number of pixels in x
    yPixels = array.shape[0]  # number of pixels in y
    pixelXSize =(xMax-xMin)/xPixels # size of the pixel in X direction     
    pixelYSize = -(yMax-yMin)/yPixels # size of the pixel in Y direction

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(fileName,xPixels,yPixels,numBands,gdal.GDT_Byte, options = [ 'PHOTOMETRIC=RGB' ])
    dataset.SetGeoTransform((xMin,pixelXSize,0,yMax,0,pixelYSize))  

    datasetSRS = osr.SpatialReference()
    datasetSRS.ImportFromEPSG(EPSGCode)
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    for i in range(0,numBands):
        dataset.GetRasterBand(i+1).WriteArray(array[:,:,i])

    dataset.FlushCache()  # Write to disk.

In [187]:
img  = np.asarray(Image.open(root_dir + 'pleineFougeres.png'))

In [188]:
EPSGCode = 4326
numBands = 3
arrayToRaster(img,root_dir + 'pleineFougeres.tif', 
              EPSGCode, 
              dted_long[idxLongMin], dted_long[idxLongMax], 
              dted_lat[idxLatMax], dted_lat[idxLatMin], 
              numBands)